In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install folium==0.2.1
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip install -q tflite-model-maker
!pip install -q tflite-support

import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 2.9 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=b9f75e2c314819d3e59c1c129612f184ea5c0989d6f428cf870e58276757de24
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 668.3 MB 14 kB/s 
     |████████████████████████████████| 462 kB 4.2 MB/s 
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
    Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
    

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/data/android_figurine.zip
!unzip -q android_figurine.zip

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'android_figurine/train',
    'android_figurine/train',
    ['android', 'pig_android']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'android_figurine/validate',
    'android_figurine/validate',
    ['android', 'pig_android']
)

In [ ]:
spec = model_spec.get('efficientdet_lite0')
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/5
255/255 [==============================] - 130s 380ms/step - det_loss: 0.7909 - cls_loss: 0.4759 - box_loss: 0.0063 - reg_l2_loss: 0.0634 - loss: 0.8543 - learning_rate: 0.0240 - gradient_norm: 2.3235 - val_det_loss: 0.9708 - val_cls_loss: 0.5943 - val_box_loss: 0.0075 - val_reg_l2_loss: 0.0644 - val_loss: 1.0352
Epoch 2/5
255/255 [==============================] - 96s 377ms/step - det_loss: 0.5218 - cls_loss: 0.3168 - box_loss: 0.0041 - reg_l2_loss: 0.0653 - loss: 0.5871 - learning_rate: 0.0275 - gradient_norm: 1.9940 - val_det_loss: 0.5521 - val_cls_loss: 0.3520 - val_box_loss: 0.0040 - val_reg_l2_loss: 0.0657 - val_loss: 0.6179
Epoch 3/5
255/255 [==============================] - 96s 376ms/step - det_loss: 0.4298 - cls_loss: 0.2641 - box_loss: 0.0033 - reg_l2_loss: 0.0658 - loss: 0.4957 - learning_rate: 0.0125 - gradient_norm: 1.9121 - val_det_loss: 0.4541 - val_cls_loss: 0.2767 - val_box_loss: 0.0035 - val_reg_l2_loss: 0.0659 - val_loss: 0.5200
Epoch 4/5
255/255 [========

In [ ]:
model.evaluate(val_data)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



--Return--
> <ipython-input-5-a8aa3030ba29>(1)<module>()->None
-> breakpoint()
<module 'tensorflow._api.v2.saved_model' from '/usr/local/lib/python3.7/dist-packages/tensorflow/_api/v2/saved_model/__init__.py'>
<bound method ObjectDetector.train of <tensorflow_examples.lite.model_maker.core.task.object_detector.ObjectDetector object at 0x7f8460adbb10>>
*** AttributeError: 'function' object has no attribute 'get_concrete_function'
*** ValueError: Expected an object of type `Trackable`, such as `tf.Module` or a subclass of the `Trackable` class, for export. Got <tensorflow_examples.lite.model_maker.core.task.object_detector.ObjectDetector object at 0x7f8460adbb10> with type <class 'tensorflow_examples.lite.model_maker.core.task.object_detector.ObjectDetector'>.
*** AttributeError: 'EfficientDetNetTrainHub' object has no attribute 'train'
*** AttributeError: 'EfficientDetNetTrainHub' object has no attribute 'infer'
['_SCALAR_UPRANKING_ON', '_TF_MODULE_IGNORED_PROPERTIES', '__call__', '__cl

In [ ]:
model.export(export_dir='.', tflite_filename='detector.tflite')

In [ ]:
model.evaluate_tflite('detector.tflite', val_data)

In [ ]:
from google.colab import files
files.download('detector.tflite')